In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
import numpy as np

import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.preprocessing import *

In [2]:
def preprocess(df):
    df2 = pd.DataFrame()
    df2['retweet'] = df.apply(lambda row: count_retweets(row['tweet']),axis=1)
    df2['mentions'] = df.apply(lambda row : count_mentions(row['tweet']),axis=1)
    df2['urls'] = df.apply(lambda row : count_urls(row['tweet']),axis=1)
    df2['hashtags'] = df.apply(lambda row : count_hashtags(row['tweet']),axis = 1)
    df['preprocess'] = df.apply(lambda row: remove_retweets(row['tweet']), axis=1)
    df['preprocess'] = df.apply(lambda row: remove_mentions(row['preprocess']), axis=1)
    df['preprocess'] = df.apply(lambda row: remove_urls(row['preprocess']), axis=1)
    df['preprocess'] = df.apply(lambda row: remove_hashtags(row['preprocess']), axis=1)
    df['preprocess'] = df.apply(lambda row: remove_additional_space(row['preprocess']), axis=1)
    df['preprocess'] = df.apply(lambda row: replace_slash_chars_by_space(row['preprocess']), axis=1)
    df['preprocess'] = df.apply(lambda row: remove_underscore(row['preprocess']), axis=1)
    df['preprocess'] = df.apply(lambda row: remove_emojis(row['preprocess']), axis=1)
    with open('../data/english_stopwords.txt', 'r') as f:
        stopwords = f.read().split('\n')
    df['preprocess'] = df.apply(lambda row: row['preprocess'].lower(), axis=1)
    to_remove_stopwords = ["not", "no", "wouldn't", "shouldn't", "couldn't", "won't", "can't", "doesn't", "isn't", "wasn't", "don't"]
    final_stopwords = [s for s in stopwords if s not in to_remove_stopwords]
    df['preprocess'] = df.apply(lambda row: remove_stopwords(row['preprocess'], final_stopwords), axis=1)
    
    return df,df2

In [3]:
data_path = "../data/"
database = "final_merged_dataset.csv"
df = pd.read_csv(data_path + database)
df['Unnamed: 0'] = [i for i in range(len(df))] #c'est moche mais ca marche c'est la vie 
df.columns = ['index', 'tweet', 'label']

df,df2 = preprocess(df)
print(df.tail())

       index                                              tweet  label  \
31996  31996  https://t.co/inx3HTRcfR Smell Diminishes by Da...      1   
31997  31997  COVID-19 outbreak on the Diamond Princess crui...      1   
31998  31998  Coronavirus China Italy | Coronavirus Outbreak...      1   
31999  31999  Coronavirus Daily Digest: April 13, 2020\nhttp...      1   
32000  32000  Detention centers are notorious for having poo...      1   

                                              preprocess  
31996        smell diminishes day 3 covid-19, study says  
31997  covid-19 outbreak diamond princess cruise ship...  
31998  coronavirus china italy | coronavirus outbreak...  
31999           coronavirus daily digest: april 13, 2020  
32000  detention centers notorious poor conditions &a...  


In [4]:
test = df2.to_numpy()

In [5]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
test = scaler.fit_transform(test)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = df['preprocess'] # use 'preprocess' for preprocessed tweets, 'tweet' for unpreprocessed

stemmer = PorterStemmer()
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

vectorizer = CountVectorizer(max_features = 100, analyzer = stemmed_words)

vectorizer.fit(corpus)
X = vectorizer.transform(corpus).toarray()
Y = df['label']

In [7]:
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer()
transformer.fit(X)
X = transformer.transform(X).toarray()

In [8]:
X

array([[0.        , 0.        , 0.        , ..., 0.55609034, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.13991225, 0.        , 0.        , ..., 0.        , 0.        ,
        0.27554131],
       [0.        , 0.91392295, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [82]:
X= np.concatenate((X,test),axis = 1)

In [83]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression


rf = RandomForestClassifier(n_estimators=200)
svm = SVC(kernel = 'linear')
nb = GaussianNB()
lr = LogisticRegression()
ada = AdaBoostClassifier(n_estimators = 200)
classifiers = [rf,svm,nb,lr,ada]

In [85]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 0)
for clf in classifiers :
    clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_test)

    print(f'Modèle {clf} : \n' + classification_report(Y_pred, Y_test))

Modèle RandomForestClassifier(n_estimators=50) : 
              precision    recall  f1-score   support

           0       0.87      0.90      0.88      3499
           1       0.92      0.90      0.91      4502

    accuracy                           0.90      8001
   macro avg       0.90      0.90      0.90      8001
weighted avg       0.90      0.90      0.90      8001

Modèle SVC(kernel='linear') : 
              precision    recall  f1-score   support

           0       0.77      0.83      0.80      3335
           1       0.87      0.82      0.85      4666

    accuracy                           0.83      8001
   macro avg       0.82      0.83      0.82      8001
weighted avg       0.83      0.83      0.83      8001

Modèle GaussianNB() : 
              precision    recall  f1-score   support

           0       0.71      0.78      0.74      3268
           1       0.84      0.78      0.81      4733

    accuracy                           0.78      8001
   macro avg       0.77 

In [14]:
from sklearn.model_selection import GridSearchCV

rf2 = RandomForestClassifier()
ada2 = AdaBoostClassifier()
params = {'n_estimators' : [20,50,100,150,200]}
clf1 = GridSearchCV(rf2,params)
clf2 = GridSearchCV(ada2,params)

In [17]:
rf_results = clf1.fit(X_train,Y_train)
ada_results = clf2.fit(X_train,Y_train)

In [18]:
rf_results.best_params_

{'n_estimators': 200}

In [19]:
ada_results.best_params_

{'n_estimators': 200}

In [10]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(Y_test, Y_pred))

[[3089  527]
 [ 397 3988]]


In [23]:
errors = pd.DataFrame({'location': [], 'text' :[], 'value': [], 'predicted': []})
for i in range(len(Y_pred)):
    if Y_test.values[i] - Y_pred[i] != 0:
        errors = errors.append({
            'location': Y_test.index[i],
            'text': df['tweet'][Y_test.index[i]],
            'preprocess': df['preprocess'][Y_test.index[i]],
            'predicted': Y_pred[i],
            'value': Y_test.values[i]
        }, ignore_index= True)
 
print(errors)

      location                                               text  value  \
0      22894.0  COVID and Sleep: Sweet Dreams Aren’t Made of T...    1.0   
1      18071.0  Seriously?\n\nWalmart to Shut Down 269 Stores,...    0.0   
2      30784.0  Is your local government doing enough testing ...    1.0   
3      30695.0  @ChuckCallesto Wearing a cloth face cover will...    1.0   
4      26966.0  I've been saying this since February: it's AIR...    1.0   
...        ...                                                ...    ...   
1250   11589.0  The positive rate of these tests is dropping a...    1.0   
1251   14358.0  Delhi Public School Selling Face Masks For Rs....    0.0   
1252   20861.0  Vitamin C protects against Coronavirus - make ...    0.0   
1253    1191.0  Israeli Researchers Near Covid-19 Vaccine Deve...    0.0   
1254    4763.0  Govt has clarified that Pradhan Mantri Suraksh...    0.0   

      predicted                                         preprocess  
0           0.0   

In [24]:
for error in errors.head().iterrows():
    print('Sample error : \n', error[1]['text'], '\n')
    print('Preprocessed error : \n', error[1]['preprocess'], '\n')
    print('Line : ', error[1]['location'])
    print('Predicted value : ', error[1]['predicted'] , '\nReal value : ', error[1]['value'], '\n')
    print('_______________________________________________\n')

Sample error : 
 COVID and Sleep: Sweet Dreams Aren’t Made of This https://t.co/wb381F0JAi 

Preprocessed error : 
 covid sleep: sweet dreams aren’t made 

Line :  22894.0
Predicted value :  0.0 
Real value :  1.0 

_______________________________________________

Sample error : 
 Seriously?

Walmart to Shut Down 269 Stores, Including 154 in US - Red Lake Nation News https://t.co/yrimsFso6m 

Preprocessed error : 
 seriously? walmart shut 269 stores, including 154 us - red lake nation news 

Line :  18071.0
Predicted value :  1.0 
Real value :  0.0 

_______________________________________________

Sample error : 
 Is your local government doing enough testing to mitigate and suppress COVID-19?

The #TTSICollaborative launches its interactive COVID Risk Map. See the most up-to-date data and read our policy recommendations and metrics for success.
https://t.co/bNny3W7bQt https://t.co/t6ttcc2UKI 

Preprocessed error : 
 local government enough testing mitigate suppress covid-19? ttsicoll